# Leakage Detection Tests

A series of tests for leakage, demonstrated on the [Los Angeles Chronic Offenders Leakage](https://github.com/dssg/la_prosecutor) project  
Requires a credentials.py file defining the following variables: dbname, user, host, password, port

In [63]:
import sys
import os
import math
import warnings

import pickle
import pandas as pd
import numpy as np
import psycopg2
from catwalk.db import connect
from catwalk.storage import FSModelStorageEngine
from sqlalchemy import create_engine
from IPython.core.interactiveshell import InteractiveShell

from credentials import dbname, user, host, password, port

pd.set_option('display.max_rows', 500)
warnings.simplefilter('ignore')
InteractiveShell.ast_node_interactivity = "all"

In [64]:
def execute_sql(statement, dbname, user, host, password, port, isolation = False, results = True):
    """
    Use psycopg2 to execute PostgreSQL queries
    
    Input:
        statement (str): SQL statement to run in database
        dbname, user, host, password (str): database credentials
        isolation (bool): indicator for whether to change isolation level to autocommit; True for queries that cannot be run 
            from within a transation (see https://wiki.postgresql.org/wiki/Psycopg2_Tutorial)
        results (bool): indicator for whether the query is expected to output results;
            for example, True for SELECT statements and False for CREATE TABLE statements
    
    Output:
        relation (dataframe): query results or empty dataframe if results = False
    """
    conn = psycopg2.connect("dbname={} user={} host={} password={}".format(dbname, user, host, password))
    cur = conn.cursor()
    if isolation:
        conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
    cur.execute(statement)
    relation = pd.DataFrame()
    if results:
        results = cur.fetchall()
        colnames = [desc[0] for desc in cur.description]
        relation = pd.DataFrame(results, columns=colnames)
    cur.close()
    conn.close()
    return relation


In [65]:
def randomize(df, do_not_randomize = None, seed=0):
    """
    Randomize column values of a file. Each column is randomized independently.
    
    Inputs:
        df (dataframe): dataframe to randomize
        do_not_randomize (list): optional list of strings indicating names of 
            columns that should not be randomized
    Outputs:
        df (dataframe): dataframe of randomized data
    """     
    df_random = df.copy()
    if do_not_randomize:
        cols = [c for c in df.columns if c not in do_not_randomize]
    else:
        cols = df.columns
        
    for col in cols:
        #print('\t\tRandomizing column ' + col)
        np.random.seed(seed)
        df_random[col] = np.random.permutation(df_random[col])

    return df_random

In [30]:
# source: https://overlaid.net/2016/02/08/replace-words-in-files-or-strings-using-python/
def do_replacement(base_text, word_map):
    """
    Helper function for replace_words_in_file. Make replacements in base_text, as 
    indicated in  word_map.
    """
    for key, val in word_map.items():
        base_text = base_text.replace(key, val)
    return base_text

def replace_words_in_file(read_from, write_to, word_map):
    """
    Create copy of a file with certain words replaced
    
    Inputs
        read_from: name of file to read from
        write_to: name of new file to be created
        word_map: dictionary of mappings between words and their replacements
            (e.g. {'old_word': 'new_word'})
    Outputs
        None. Will create a new file with the name given in write_to
        
    """
    print('Generating file ', write_to)
    # Open your desired file as 't' and read the lines into string 'tempstr'
    t = open(read_from, 'r')
    tempstr = t.read()
    t.close()

    # Using the "replace_words" function, we'll pass in our tempstr to be used as the base, 
    # and our device_values to be used as replacement.  
    output = do_replacement(tempstr, word_map)

    # Write out the new config file
    fout = open(write_to, 'w')
    fout.write(output)
    fout.close()

In [31]:
def get_new_filename(filename, suffix, replacement_map=None):
    """
    Output a new filename (str) given a filename and suffix to append. Assumes file extension at the end is separated by a period.
    """
    k = filename.rfind(".")
    new_filename = filename[:k] + suffix + '.' + filename[k+1:]
    if replacement_map:
        new_filename = do_replacement(new_filename, replacement_map)
    return new_filename

## I. Randomize the input data

A number of preprocessing steps were required for this project, because raw cases and bookings data did not come with unique identifiers for the individuals involved. As such, we cannot randomize the raw data because we would be unable to identify individuals in the results. Randomization must be done after entity linkage but before features are built.

Pull the information schema from selected database.

In [32]:
# name of the database schema containing input data to be randomized
INPUT_SCHEMA = 'staging' 

In [33]:
statement = "SELECT * FROM information_schema.tables;"
tables = execute_sql(statement, dbname, user, host, password, port)

Select the schema containing input datasets to be randomized. For this project, the schema is named staging

In [34]:
input_tables = tables.table_name[tables.table_schema == INPUT_SCHEMA]
input_tables = list(input_tables)

First created the _randomized schema if it doesn't yet exist

In [35]:
statement = "CREATE SCHEMA IF NOT EXISTS {}_randomized;".format(INPUT_SCHEMA)
output = execute_sql(statement, dbname, user, host, password, port, isolation=True, results = False)

Copy over all the tables from original input schema, i.e. all tables in input_tables list.

Randomize only a subset of these tables. Tables to be randomized are:    
1) The set of tables from the staging schema that are referenced in the from\_obj section of each of the yaml files in /config/features. These staging tables have been linked to entity identifiers and are the source data for feature generation.  
2) Two labels tables from the staging schema that are referenced in config files: 'labels_casesonly_multiprior_win6mo_lab6mo' and 'labels_multiprior_win6mo_lab6mo'. Only randomizing the outcomes column in these labels tables to maintain the entity_id and outcome_date key pairing. This way, the states table does not need to be rerun.

Note that table names beginning with "staging.entity_" are not included in the list because they contain mappings between entity identifiers and other data. These relationships must be maintained so should not be randomized.

Write output to _randomized schema. 

In [36]:
input_tables

['addinfo',
 'booking',
 'booking_addl_info',
 'case_booking',
 'cases',
 'casesaka',
 'casescharge',
 'casesflag',
 'ccms_to_ucc',
 'citation',
 'docass',
 'entity_address',
 'entity_aka',
 'entity_all_events',
 'entity_any_name',
 'entity_best_demos',
 'entity_booking',
 'entity_case',
 'entity_cii',
 'entity_dmv',
 'entity_dob',
 'entity_gender',
 'entity_main_number',
 'entity_name',
 'entity_race',
 'feature_case_info',
 'feature_case_results',
 'feature_casesflag_info',
 'feature_charge_info',
 'firstname_gender',
 'firstname_race',
 'labels_1prior_win6mo_lab6mo',
 'labels_casesonly_multiprior_win6mo_lab6mo',
 'labels_multiprior_win6mo_lab6mo',
 'lastname_race',
 'prob',
 'probcon',
 'states_win6mo_lab6mo',
 'status_h',
 'turkey_list',
 'charge_categories',
 'case_flag_lkup',
 'case_dispo_lkup',
 'orig_agency_lkup',
 'branch_lkup',
 'case_result_lkup',
 'charge_lkup']

In [37]:
input_tables_to_randomize = [
               'booking_addl_info', 
               'branch_lkup',
               'case_booking',  
               'case_dispo_lkup', 
               'case_flag_lkup',
               'case_result_lkup',
               'charge_lkup',
               'feature_casesflag_info', 
               'feature_case_info', 
               'feature_case_results',
               'feature_charge_info',
               'labels_casesonly_multiprior_win6mo_lab6mo',
               'orig_agency_lkup',
               'labels_multiprior_win6mo_lab6mo'
               ]

In [40]:
for table_name in input_tables:
    seed = 0 # initiate to 0, increment in each iteration
    
    print("Working on table {}".format(table_name))
    
    if table_name in input_tables_to_randomize: # randomize before adding to _randomized schema
        # Pull the table from original schema
        print("\tPulling table")
        statement = "SELECT * FROM {}.{};".format(INPUT_SCHEMA, table_name)
        table = execute_sql(statement, dbname, user, host, password, port)

        # Randomize the table
        print("\tRandomizing")
        if table_name in ['labels_casesonly_multiprior_win6mo_lab6mo','labels_multiprior_win6mo_lab6mo']:
            randomized_table = randomize(table, do_not_randomize=['entity_id', 'outcome_date'], seed = seed)          
        else:
            randomized_table = randomize(table, seed = seed)

        # Make a new table in _randomized schema
        print("\tUploading randomized version")
        statement = "CREATE TABLE IF NOT EXISTS {0}_randomized.{1}_randomized (LIKE {0}.{1} INCLUDING ALL);".format(INPUT_SCHEMA, table_name)
        output = execute_sql(statement, dbname, user, host, password, port, isolation = True, results = False)

        # Write results into new table
        statement = "SELECT COUNT(*) FROM {}_randomized.{}_randomized;".format(INPUT_SCHEMA, table_name)
        output = execute_sql(statement, dbname, user, host, password, port)
        if output.iloc[0,0]>0: # do nothing if new table already contains data
            print("\t*****SKIPPING TABLE {}_randomized -- it already has data".format(table_name))
        else:
            engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(user, password, host, port, dbname))
            randomized_table.to_sql(table_name+'_randomized', engine, schema = '{}_randomized'.format(INPUT_SCHEMA), index = False, if_exists='append')
    
    else: # copy to _randomized schema as is, without randomizing
        # Make a new table in _randomized schema
        print("\tUploading original version without randomizing")
        statement = "CREATE TABLE IF NOT EXISTS {0}_randomized.{1} (LIKE {0}.{1} INCLUDING ALL);".format(INPUT_SCHEMA, table_name)
        output = execute_sql(statement, dbname, user, host, password, port, isolation = True, results = False)
        
        # Copy original data without randomizing
        statement = "SELECT COUNT(*) FROM {}_randomized.{};".format(INPUT_SCHEMA, table_name)
        output = execute_sql(statement, dbname, user, host, password, port)
        if output.iloc[0,0]>0: # do nothing if new table already contains data
            print("\t*****SKIPPING TABLE {} -- it already has data".format(table_name))
        else:
            statement = "INSERT INTO {0}_randomized.{1} (SELECT * from {0}.{1});".format(INPUT_SCHEMA, table_name)
            output = execute_sql(statement, dbname, user, host, password, port, isolation = True, results = False)
    seed += 1

Working on table addinfo
	Uploading original version without randomizing
	*****SKIPPING TABLE addinfo -- it already has data
Working on table booking
	Uploading original version without randomizing
	*****SKIPPING TABLE booking -- it already has data
Working on table booking_addl_info
	Pulling table
	Randomizing
	Uploading randomized version
	*****SKIPPING TABLE booking_addl_info_randomized -- it already has data
Working on table case_booking
	Pulling table
	Randomizing
	Uploading randomized version
	*****SKIPPING TABLE case_booking_randomized -- it already has data
Working on table cases
	Uploading original version without randomizing
	*****SKIPPING TABLE cases -- it already has data
Working on table casesaka
	Uploading original version without randomizing
	*****SKIPPING TABLE casesaka -- it already has data
Working on table casescharge
	Uploading original version without randomizing
	*****SKIPPING TABLE casescharge -- it already has data
Working on table casesflag
	Uploading original 

Also randomized the labels, table labels_multiprior_win6mo_lab6mo; forgot to include in input_tables_to_randomize list when previous cell was run

## II. Edit config files so that they point to the newly randomized raw schema

Experiment config file being used on the project is experiment_config_multiprior.yaml. Also need to edit all the feature config files under config/features. New versions of config files with an "_edited" suffix will be produced and saved in "features_randomized" folder.

In [ ]:
CONFIG_FILES = [
    'config/experiment_config_multiprior.yaml', 
    'config/features/booking_info.yaml',
    'config/features/case_flags.yaml',
    'config/features/case_info.yaml',
    'config/features/case_results.yaml',
    'config/features/charge_info.yaml',
    'config/features/days_since.yaml',
    'config/features/demos.yaml',
    'config/features/multi_prior.yaml'
    ]

Make a dictionary of text replacements to apply across every preprocessing & config file. This should include:
- Schema and table names that have been changed, i.e. have a "_randomized" suffix

In [41]:
WORD_MAP = {x:x+'_randomized' for x in input_tables_to_randomize}
WORD_MAP[INPUT_SCHEMA+'.'] = INPUT_SCHEMA+'_randomized.'
WORD_MAP

{'booking_addl_info': 'booking_addl_info_randomized',
 'branch_lkup': 'branch_lkup_randomized',
 'case_booking': 'case_booking_randomized',
 'case_dispo_lkup': 'case_dispo_lkup_randomized',
 'case_flag_lkup': 'case_flag_lkup_randomized',
 'case_result_lkup': 'case_result_lkup_randomized',
 'charge_lkup': 'charge_lkup_randomized',
 'feature_case_info': 'feature_case_info_randomized',
 'feature_case_results': 'feature_case_results_randomized',
 'feature_casesflag_info': 'feature_casesflag_info_randomized',
 'feature_charge_info': 'feature_charge_info_randomized',
 'labels_casesonly_multiprior_win6mo_lab6mo': 'labels_casesonly_multiprior_win6mo_lab6mo_randomized',
 'labels_multiprior_win6mo_lab6mo': 'labels_multiprior_win6mo_lab6mo_randomized',
 'orig_agency_lkup': 'orig_agency_lkup_randomized',
 'staging.': 'staging_randomized.'}

Apply find+replace to every config file. Write feature config files to a new folder (config/features_randomized) that will later be referenced when running the experiment. 

In [ ]:
# !cd config && mkdir features_randomized

In [ ]:
#for filename in CONFIG_FILES:
#    new_filename = get_new_filename(filename, "_edited", {'features': 'features_randomized'})
#    replace_words_in_file(filename, new_filename, WORD_MAP) 

**NOTE**: Be sure to open the outputted files (with "_edited" suffix) to check that no unexpected replacements were made and that all necessary replacements are made. 

Model Group Key: We have to manually add a "purpose" model group key into the experiment config file. Including a "purpose" indicates that this experiment is for leakage detection and helps to distinguish it from existing experiments. Add "purpose: leakage_detection" under the user_metadata section, and add "purpose" under the model_group_keys section.

## III. Run an experiment with this new setup

Now that we've set up randomized versions of the original schemas in the database and created a new config file for a randomized experiment, it's time to run the actual triage experiment. This is done in an aws ec2 instance.

Make sure you have a database.yaml file, required by triage for modeling. See example [here](https://github.com/dssg/la_prosecutor/blob/master/example_database.yaml).

Update run.py to reflect the particulars of this new config.
- set PROJECT_PATH to the local directory where you want to store output
- set n_processes to be less than the number of cores on your machine

Run the following command: python run.py -v -c config/experiment_config_multiprior_edited.yaml -f features_randomized > log.txt

## IV. Align the results of randomized experiment against original experiment

Model group from the original, non-randomized experiment: 22  
Model group from randomized experiment: 115. 

Note: experiment didn't finish running successfully, so for now we can only look at model_id 1116, 1120, 1124, and 1128 under model_group_id 115, with train_end_time between 2012-07-01 and 2014-01-01. This corresponds to models 22, 50, 76, and 100 under model_group_22.

First look at model groups

In [66]:
GROUP_ID_ORIG = 22
GROUP_ID_RAND = 115

In [67]:
# original model group
statement = "SELECT * FROM results.models where model_group_id = {};".format(GROUP_ID_ORIG)
models_orig = execute_sql(statement, dbname, user, host, password, port)
models_orig

,model_id,model_group_id,model_hash,run_time,batch_run_time,model_type,model_parameters,model_comment,batch_comment,config,experiment_hash,train_end_time,test,train_matrix_uuid,train_label_window
0,22,22,77f84946bf25e4e5ef2879cc126c45ee,2017-09-26 22:06:39.349441,2017-09-26 22:06:31.322862,catwalk.estimators.classifiers.ScaledLogisticR...,"{'penalty': 'l1', 'C': 1}",None,None,None,f58631bba52780245ce2b57e181b97a5,2012-07-01,False,7f857d499d4cb5f198ada6fbfc298932,180 days
1,50,22,b1c57c7b607cbbb48a5ad1b8eed22678,2017-09-27 01:03:23.889844,2017-09-27 01:03:17.143301,catwalk.estimators.classifiers.ScaledLogisticR...,"{'penalty': 'l1', 'C': 1}",None,None,None,f58631bba52780245ce2b57e181b97a5,2013-01-01,False,9abc62d7bb8ba6f76b159e42978db856,180 days
2,76,22,e745ecd44c0808091c83a1ac92ec5e84,2017-09-27 04:13:12.106100,2017-09-27 04:13:04.753935,catwalk.estimators.classifiers.ScaledLogisticR...,"{'penalty': 'l1', 'C': 1}",None,None,None,f58631bba52780245ce2b57e181b97a5,2013-07-01,False,3c24fb375687ee858081c270c2e2523a,180 days
3,100,22,0b86a9335051571a1abb7b0281b3afa4,2017-09-27 07:14:26.207931,2017-09-27 07:14:17.470914,catwalk.estimators.classifiers.ScaledLogisticR...,"{'penalty': 'l1', 'C': 1}",None,None,None,f58631bba52780245ce2b57e181b97a5,2014-01-01,False,1abb834182a2c7d2813571290db4092c,180 days
4,127,22,b1f2449b07561c3656a2f196a6ff5b40,2017-09-27 10:40:08.474958,2017-09-27 10:40:00.779670,catwalk.estimators.classifiers.ScaledLogisticR...,"{'penalty': 'l1', 'C': 1}",None,None,None,f58631bba52780245ce2b57e181b97a5,2014-07-01,False,a8d2f56c60a0eb0c6817a556ea04ad01,180 days
5,154,22,4377998afedbf7eaeecfa845ce594fa7,2017-09-27 14:20:25.823376,2017-09-27 14:20:17.907213,catwalk.estimators.classifiers.ScaledLogisticR...,"{'penalty': 'l1', 'C': 1}",None,None,None,f58631bba52780245ce2b57e181b97a5,2015-01-01,False,068f2995bfd9084867566271e0ed028b,180 days
6,180,22,539f1a5a91f898eb10e87215c1a9d075,2017-09-27 17:53:51.344478,2017-09-27 17:53:40.796521,catwalk.estimators.classifiers.ScaledLogisticR...,"{'penalty': 'l1', 'C': 1}",None,None,None,f58631bba52780245ce2b57e181b97a5,2015-07-01,False,13b7167eaa4f276d2cc2304ca11bf31b,180 days
7,205,22,5c225133baad5556244fe722fe098bdf,2017-09-27 21:28:12.925761,2017-09-27 21:28:05.338159,catwalk.estimators.classifiers.ScaledLogisticR...,"{'penalty': 'l1', 'C': 1}",None,None,None,f58631bba52780245ce2b57e181b97a5,2016-01-01,False,760c7a17a6800bffc6616d6c0dd4f611,180 days
8,232,22,ca28b4335551d6e75a332329d876a340,2017-09-28 01:08:52.532772,2017-09-28 01:08:43.338539,catwalk.estimators.classifiers.ScaledLogisticR...,"{'penalty': 'l1', 'C': 1}",None,None,None,f58631bba52780245ce2b57e181b97a5,2016-07-01,False,b0db98ce151fdeec11f0eeaea061d56f,180 days
9,258,22,564296d9b3353ba25b507b0dff835085,2017-09-28 04:59:54.356953,2017-09-28 04:59:46.265510,catwalk.estimators.classifiers.ScaledLogisticR...,"{'penalty': 'l1', 'C': 1}",None,None,None,f58631bba52780245ce2b57e181b97a5,2017-01-01,False,740dce77bc130c0a469b5bd46bb4aa35,180 days


In [70]:
# randomized model group 
statement = "SELECT * FROM results.models WHERE model_group_id = {}".format(GROUP_ID_RAND)
model_groups = execute_sql(statement, dbname, user, host, password, port)
model_groups

,model_id,model_group_id,model_hash,run_time,batch_run_time,model_type,model_parameters,model_comment,batch_comment,config,experiment_hash,train_end_time,test,train_matrix_uuid,train_label_window
0,1116,115,eee5cc5eea5644ac159505adc5d42274,2018-05-10 02:55:53.280750,2018-05-10 02:52:52.360614,catwalk.estimators.classifiers.ScaledLogisticR...,"{'penalty': 'l1', 'C': 1}",None,None,None,1eeafc4c287077d7574d1e71cc2072c6,2012-07-01,False,dd8722509b05f139a3760e3bb654781c,180 days
1,1120,115,a85342831d5217f90fd56a4725eb6845,2018-05-10 03:14:59.283845,2018-05-10 03:11:59.295543,catwalk.estimators.classifiers.ScaledLogisticR...,"{'penalty': 'l1', 'C': 1}",None,None,None,1eeafc4c287077d7574d1e71cc2072c6,2013-01-01,False,af1fcbe4361b4a373167637902bf7f9f,180 days
2,1124,115,e41295be7196d7a99c8904be4868d53e,2018-05-10 03:34:55.970298,2018-05-10 03:31:50.419180,catwalk.estimators.classifiers.ScaledLogisticR...,"{'penalty': 'l1', 'C': 1}",None,None,None,1eeafc4c287077d7574d1e71cc2072c6,2013-07-01,False,77500a566766e07fa10ead975946ba65,180 days
3,1128,115,0c1403fb84acc6902cb815eb1559aebe,2018-05-10 03:55:02.977249,2018-05-10 03:51:50.127878,catwalk.estimators.classifiers.ScaledLogisticR...,"{'penalty': 'l1', 'C': 1}",None,None,None,1eeafc4c287077d7574d1e71cc2072c6,2014-01-01,False,e6a0eff5c0697f65fc1b6daf77f36df4,180 days
4,1132,115,a13fedb30582477920eafacf8a78ffdb,2018-05-10 04:15:31.157220,2018-05-10 04:12:15.831237,catwalk.estimators.classifiers.ScaledLogisticR...,"{'penalty': 'l1', 'C': 1}",None,None,None,1eeafc4c287077d7574d1e71cc2072c6,2014-07-01,False,4efe5cb3ce4c3323c0aa3c4b611f3862,180 days


Now look at models

In [172]:
MODEL_ID_ORIG = 22 
MODEL_ID_RAND = 1116 

In [173]:
statement = "SELECT * FROM results.models where model_id = {};".format(MODEL_ID_ORIG)
models_rand = execute_sql(statement, dbname, user, host, password, port)
models_rand

,model_id,model_group_id,model_hash,run_time,batch_run_time,model_type,model_parameters,model_comment,batch_comment,config,experiment_hash,train_end_time,test,train_matrix_uuid,train_label_window
0,22,22,77f84946bf25e4e5ef2879cc126c45ee,2017-09-26 22:06:39.349441,2017-09-26 22:06:31.322862,catwalk.estimators.classifiers.ScaledLogisticR...,"{'penalty': 'l1', 'C': 1}",None,None,None,f58631bba52780245ce2b57e181b97a5,2012-07-01,False,7f857d499d4cb5f198ada6fbfc298932,180 days


In [174]:
statement = "SELECT * FROM results.models where model_id = {};".format(MODEL_ID_RAND)
models_rand = execute_sql(statement, dbname, user, host, password, port)
models_rand

,model_id,model_group_id,model_hash,run_time,batch_run_time,model_type,model_parameters,model_comment,batch_comment,config,experiment_hash,train_end_time,test,train_matrix_uuid,train_label_window
0,1116,115,eee5cc5eea5644ac159505adc5d42274,2018-05-10 02:55:53.280750,2018-05-10 02:52:52.360614,catwalk.estimators.classifiers.ScaledLogisticR...,"{'penalty': 'l1', 'C': 1}",None,None,None,1eeafc4c287077d7574d1e71cc2072c6,2012-07-01,False,dd8722509b05f139a3760e3bb654781c,180 days


Now look at predictions. Why does randomized model give the same score for every single person?

In [175]:
statement = "SELECT * FROM results.predictions where model_id = {};".format(MODEL_ID_ORIG)
predictions_orig = execute_sql(statement, dbname, user, host, password, port)
predictions_orig.head()

,model_id,entity_id,as_of_date,score,label_value,rank_abs,rank_pct,matrix_uuid,test_label_window
0,22,1,2012-07-01,0.012619326341990676,0,None,None,1dfbc6b3ed8719079d8ccb54160726a1,180 days
1,22,2,2012-07-01,0.025366407080473292,1,None,None,1dfbc6b3ed8719079d8ccb54160726a1,180 days
2,22,4,2012-07-01,0.021707432366426957,0,None,None,1dfbc6b3ed8719079d8ccb54160726a1,180 days
3,22,6,2012-07-01,0.007842802182639242,0,None,None,1dfbc6b3ed8719079d8ccb54160726a1,180 days
4,22,32,2012-07-01,0.04150020718499921,0,None,None,1dfbc6b3ed8719079d8ccb54160726a1,180 days


In [176]:
statement = "SELECT * FROM results.predictions where model_id = {};".format(MODEL_ID_RAND)
predictions_rand = execute_sql(statement, dbname, user, host, password, port)
predictions_rand.head(100)

,model_id,entity_id,as_of_date,score,label_value,rank_abs,rank_pct,matrix_uuid,test_label_window
0,1116,1,2012-07-01,0.046078318465156316,0,None,None,ae5dfa11a6dc725a5ea9058a99eb599b,180 days
1,1116,2,2012-07-01,0.046078318465156316,0,None,None,ae5dfa11a6dc725a5ea9058a99eb599b,180 days
2,1116,4,2012-07-01,0.046078318465156316,0,None,None,ae5dfa11a6dc725a5ea9058a99eb599b,180 days
3,1116,6,2012-07-01,0.046078318465156316,0,None,None,ae5dfa11a6dc725a5ea9058a99eb599b,180 days
4,1116,32,2012-07-01,0.046078318465156316,0,None,None,ae5dfa11a6dc725a5ea9058a99eb599b,180 days
5,1116,61,2012-07-01,0.046078318465156316,0,None,None,ae5dfa11a6dc725a5ea9058a99eb599b,180 days
6,1116,75,2012-07-01,0.046078318465156316,0,None,None,ae5dfa11a6dc725a5ea9058a99eb599b,180 days
7,1116,90,2012-07-01,0.046078318465156316,0,None,None,ae5dfa11a6dc725a5ea9058a99eb599b,180 days
8,1116,105,2012-07-01,0.046078318465156316,0,None,None,ae5dfa11a6dc725a5ea9058a99eb599b,180 days
9,1116,118,2012-07-01,0.046078318465156316,0,None,None,ae5dfa11a6dc725a5ea9058a99eb599b,180 days


In [177]:
np.mean(predictions_orig.label_value)
np.mean(predictions_rand.label_value)

0.05574758806169508

0.045672199437418133

Why are there different number of observations in the two predictions tables?   
Answer: Entity IDs have been reassigned since then, so it's okay that they don't match. However, the overall length of the lists of people should be the same, because they depend on the entity states which have not changed. Going to rerun staging pipeline to check for instabilities and investigate why prediction counts of original model do not match the state.

For example: 

Model with train end time 2012-07-01:  
Only 87,353 entities in common between original and random predictions.  
There are 245,056 entities in original and not in random.  
There are 238,644 entities in random and not in original.

Model with train end time 2013-07-01:  
Only 93,010 entities in common between original and random predictions.  
There are 251,345 entities in original and not in random.  
There are 244,662 entities in random and not in original.

In [178]:
predictions_orig.shape
predictions_rand.shape

(332409, 9)

(325997, 9)

In [179]:
entities_orig = set(predictions_orig.entity_id)
entities_rand = set(predictions_rand.entity_id)
entities_diff1 = entities_orig - entities_rand
entities_diff2 = entities_rand - entities_orig

In [180]:
len(entities_diff1)
len(entities_diff2)

245056

238644

In [181]:
len(entities_orig.intersection(entities_rand))

87353

Compare train and test matrices between original and randomized model. Issues:   
- There are additional feature columns in original model that are not in randomized model. Ivan/Erika have been working on fixing this.
- Some features do not seem to have been created correctly in randomized model, such as the days_since... column which is only 0.

In [109]:
train_matrix_orig = pd.read_csv('orig_models/7f857d499d4cb5f198ada6fbfc298932.csv', nrows=5000)
train_orig_cols = set(train_matrix_orig.columns)

In [110]:
train_matrix_orig.shape
train_matrix_orig.describe()

(5000, 3298)

,entity_id,days_since_entity_id_50y_last_event_min,booking_info_entity_id_1y_attempt_flag_max,booking_info_entity_id_1y_attempt_flag_sum,booking_info_entity_id_1y_unknown_charge_max,booking_info_entity_id_1y_unknown_charge_sum,booking_info_entity_id_1y_always_infraction_max,booking_info_entity_id_1y_always_infraction_sum,booking_info_entity_id_1y_always_infraction_avg,booking_info_entity_id_1y_always_misdemeanor_max,...,num_prior_entity_id_10y_bookings_sum,num_prior_entity_id_10y_any_sum,num_prior_entity_id_10y_nonexcl_sum,num_prior_entity_id_10y_nonexcl_avg,num_prior_entity_id_50y_cases_sum,num_prior_entity_id_50y_bookings_sum,num_prior_entity_id_50y_any_sum,num_prior_entity_id_50y_nonexcl_sum,num_prior_entity_id_50y_nonexcl_avg,outcome
count,5000.000000,5000.000000,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,...,5000.0,5000.000000,5000.000000,5000.000000,5000.00000,5000.0,5000.00000,5000.000000,5000.000000,5000.000000
mean,12217.154400,2639.985600,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.440800,1.437400,0.684802,2.82600,0.0,2.82600,2.819400,0.996677,0.026000
std,4902.399322,1701.718874,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.555564,1.552728,0.464006,1.79465,0.0,1.79465,1.790817,0.046351,0.159151
min,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.00000,0.000000,0.000000,0.000000
25%,8205.750000,1104.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,2.00000,0.0,2.00000,2.000000,1.000000,0.000000
50%,12248.500000,2501.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.000000,1.000000,1.000000,2.00000,0.0,2.00000,2.000000,1.000000,0.000000
75%,16470.000000,4104.250000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.000000,2.000000,1.000000,3.00000,0.0,3.00000,3.000000,1.000000,0.000000
max,20475.000000,6207.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,21.000000,21.000000,1.000000,28.00000,0.0,28.00000,28.000000,1.000000,1.000000


In [107]:
train_matrix_rand = pd.read_csv('experiment_output/matrices/ae5dfa11a6dc725a5ea9058a99eb599b.csv', nrows=5000)
train_rand_cols = set(train_matrix_rand.columns)

In [101]:
train_matrix_rand.shape
train_matrix_rand.describe()

(100, 3287)

,entity_id,case_info_entity_id_1y_num_akas_to_date_max,case_info_entity_id_1y_any_prior_addinfo_max,case_info_entity_id_1y_any_hosp_addinfo_max,case_info_entity_id_1y_num_charges_max,case_info_entity_id_1y_num_charges_sum,case_info_entity_id_1y_num_charges_avg,case_info_entity_id_1y_transient_address_flag_max,case_info_entity_id_1y_transient_address_flag_sum,case_info_entity_id_1y_transient_address_flag_avg,...,booking_info_entity_id_50y_always_violent_max,booking_info_entity_id_50y_always_violent_sum,booking_info_entity_id_50y_always_violent_avg,booking_info_entity_id_50y_ever_serious_max,booking_info_entity_id_50y_ever_serious_sum,booking_info_entity_id_50y_ever_serious_avg,booking_info_entity_id_50y_always_serious_max,booking_info_entity_id_50y_always_serious_sum,booking_info_entity_id_50y_always_serious_avg,outcome
count,100.000000,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.000000
mean,1324.620000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.020000
std,682.063518,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.140705
min,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
25%,843.750000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
50%,1442.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
75%,1799.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
max,2416.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000


In [140]:
train_matrix_orig.days_since_entity_id_50y_last_event_min.describe()


count    5000.000000
mean     2639.985600
std      1701.718874
min         0.000000
25%      1104.500000
50%      2501.500000
75%      4104.250000
max      6207.000000
Name: days_since_entity_id_50y_last_event_min, dtype: float64

In [117]:
train_matrix_rand.days_since_entity_id_50y_last_event_min.describe()

count    5000.0
mean        0.0
std         0.0
min         0.0
25%         0.0
50%         0.0
75%         0.0
max         0.0
Name: days_since_entity_id_50y_last_event_min, dtype: float64

In [113]:
diff1 = train_orig_cols - train_rand_cols
diff1

diff2 = train_rand_cols - train_orig_cols
diff2

{'charge_info_entity_id_10y_chrg_cat_id_29_max',
 'charge_info_entity_id_10y_chrg_cat_id_29_sum',
 'charge_info_entity_id_1y_chrg_cat_id_29_max',
 'charge_info_entity_id_1y_chrg_cat_id_29_sum',
 'charge_info_entity_id_2y_chrg_cat_id_29_max',
 'charge_info_entity_id_2y_chrg_cat_id_29_sum',
 'charge_info_entity_id_50y_chrg_cat_id_29_max',
 'charge_info_entity_id_50y_chrg_cat_id_29_sum',
 'charge_info_entity_id_5y_chrg_cat_id_29_max',
 'charge_info_entity_id_5y_chrg_cat_id_29_sum',
 'freq_chgs_entity_id_7d_max_cat_id_29_max'}

set()

Grab the evaluation metrics for each of these models.

In [183]:
statement = "SELECT * FROM results.evaluations where model_id = {};".format(MODEL_ID_ORIG)
evaluations_orig = execute_sql(statement, dbname, user, host, password, port)
evaluations_orig.head()

,model_id,evaluation_start_time,evaluation_end_time,example_frequency,metric,parameter,value,num_labeled_examples,num_labeled_above_threshold,num_positive_labels,sort_seed
0,22,2012-07-01,2012-07-01,1 days,fpr@,1000_abs,0.0011947317110469673,332409,1000,18531,1506473051
1,22,2012-07-01,2012-07-01,1 days,fpr@,10.0_pct,0.08083714054505253,332409,33240,18531,1506473051
2,22,2012-07-01,2012-07-01,1 days,fpr@,1.0_pct,0.005279121187212866,332409,3324,18531,1506473051
3,22,2012-07-01,2012-07-01,1 days,fpr@,1500_abs,0.0019944054696410707,332409,1500,18531,1506473051
4,22,2012-07-01,2012-07-01,1 days,fpr@,150_abs,0.0001083223418015917,332409,150,18531,1506473051


In [182]:
statement = "SELECT * FROM results.evaluations where model_id = {};".format(MODEL_ID_RAND)
evaluations_rand = execute_sql(statement, dbname, user, host, password, port)
evaluations_rand.head()

,model_id,evaluation_start_time,evaluation_end_time,example_frequency,metric,parameter,value,num_labeled_examples,num_labeled_above_threshold,num_positive_labels,sort_seed
0,1116,2012-07-01,2012-07-01,1 days,fpr@,1000_abs,0.0030728878717358604,325997,1000,14889,1525921583
1,1116,2012-07-01,2012-07-01,1 days,fpr@,10.0_pct,0.09995242809570953,325997,32599,14889,1525921583
2,1116,2012-07-01,2012-07-01,1 days,fpr@,1.0_pct,0.009993314218856474,325997,3259,14889,1525921583
3,1116,2012-07-01,2012-07-01,1 days,fpr@,1500_abs,0.004618974761176183,325997,1500,14889,1525921583
4,1116,2012-07-01,2012-07-01,1 days,fpr@,150_abs,0.0004692904071897862,325997,150,14889,1525921583
